# Sentiment analysis from movie reviews
### The data used in this project consists of user-generated movie reviews and classification of whether the user liked the movie or not based on its associated rating
### The data used is imdb movie reviews from keras dataset
### RNN will be used to perform the sentiment analysis on full-text movie reviews.
### since understanding written language requires keeping track of all the words in a sentence. we need a recurrent neural network to keep a "memmory" of the words that have come before as it "reads" sentences over time
### In particular. we'll use LSTM(Long Short-Term Memory) cells because we don't really want to "forget" words too quickly, words early on in a sentence can affect the meaning of that sentence significantly 
### The next things is to import all the necessary dependency 

In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

### To import our training and testing data, we specify that we only care about the 20,000 most popular words in the dataset in order to keep things somewhat manageable. The dataset includes 5,000 training reviews and 25,000 testing reviews for some reason 

In [2]:
print("Loading data.................")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Loading data.................
17473536/17464789 [==============================] - 0s 0us/step


### Let's get a feel for what this data looks like. let's look at the first training feature. which should represent a written movie review

In [3]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

### That doesn't look like a movie review But this data set has spared us a lot of trouble. They have already converted words to integer-based indices. The actual letters that make up a word don't really matter as far as our model is concerned, what matters are the words themselves and our model needs numbers to work with not letter
### so just keep in mind that each number in the training features represent some specific words. it's a bummer that we can't just read the reviews in English as a gut check to see if sentiment analysis is really working, though 

### what do the labels look like

In [4]:
y_train[0]

1

### They are just 0 or 1, which indicates whether the reviewer said they liked the movie or not
### So to recap, we have a bunch of movie reviews that have been converted into vectors of words represented by integers, and a binary sentiment classification to learn from 
### RNN's can blow up quickly, so again to keep things manageable on our little PC let's limit the reviews to their first 80 words

In [5]:
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

### Now let's set up our neural network model, considering how complicated a LSTM recurrent neural network is under the hood, it's really amazing how easy this is to do with keras
### we will start with an Embedding layer-this is just a step that converts the input data into dense vectors of fixed size that's better suited for a neural network. You generally see this in conjunction with index-based text data like we have here. The 20,000 indicates the vocabulary size (remember we said we only wanted the top 20,000 words) and 128 is the output dimension of 128 units 
### Next we just have to set up a LSTM layer for the RNN itself. It's that easy. We specify 128 to match the output size of the Embedding layer and dropout terms to avoid overfitting which RNN's are particularly prone to 
### Finally we just need to boil it down to a single neuron with a sigmoid activation function to choose our binary sentiment classification of 0 or 1

In [6]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

### As this is a binary classification problem, we'll use the binary_crossentropy loss function. And the Adam optimizer is usually a good choice(we can try others too) 

In [7]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

### Now we will actually train our model RNN's like CNN's are very resource heavy. Keeping the batch size relatively small is the key to enabling this to run on PC at all in the real word of course. we'd be taking advantage of GPU's installed across many computers on a cluster to make this scale a lot better
# Warning
### This will take a very long time to run even on a fast PC, don't execute the next block unless you're prepared to tie up your computer for an hour or more

In [8]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=2,
          validation_data = (x_test, y_test))

Epoch 1/15
782/782 - 236s - loss: 0.4301 - accuracy: 0.7956 - val_loss: 0.3803 - val_accuracy: 0.8315 - 236s/epoch - 301ms/step
Epoch 2/15
782/782 - 231s - loss: 0.2539 - accuracy: 0.8999 - val_loss: 0.3875 - val_accuracy: 0.8305 - 231s/epoch - 295ms/step
Epoch 3/15
782/782 - 228s - loss: 0.1634 - accuracy: 0.9383 - val_loss: 0.4553 - val_accuracy: 0.8304 - 228s/epoch - 292ms/step
Epoch 4/15
782/782 - 229s - loss: 0.1069 - accuracy: 0.9618 - val_loss: 0.5034 - val_accuracy: 0.8216 - 229s/epoch - 292ms/step
Epoch 5/15
782/782 - 231s - loss: 0.0759 - accuracy: 0.9740 - val_loss: 0.6242 - val_accuracy: 0.8158 - 231s/epoch - 295ms/step
Epoch 6/15
782/782 - 227s - loss: 0.0557 - accuracy: 0.9815 - val_loss: 0.6776 - val_accuracy: 0.8154 - 227s/epoch - 291ms/step
Epoch 7/15
782/782 - 227s - loss: 0.0322 - accuracy: 0.9893 - val_loss: 0.9484 - val_accuracy: 0.8132 - 227s/epoch - 291ms/step
Epoch 8/15
782/782 - 227s - loss: 0.0336 - accuracy: 0.9891 - val_loss: 0.8798 - val_accuracy: 0.8180 - 

### Let's evaluate our model's accuracy

In [9]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32,
                            verbose=2)

print("Test score: ", score)
print("Test accuracy: ", acc)

782/782 - 38s - loss: 1.0186 - accuracy: 0.8051 - 38s/epoch - 48ms/step
Test score:  1.0186083316802979
Test accuracy:  0.8051199913024902


### The test accuracy score is not too bad but it can be better than. the data used here has been limited to 80 words of each review
### We have just made a neural network that can "read" reviews and deduce whether the author liked the movie or not based on that text. And it takes the context of each word and its position in the review into account and setting up the model itself was just a few lines of code, it's pretty incredible what can be done using keras 